# 基于 ERNIE 4.5-0.3B 的大语言模型预训练教程

## 1. 引言

欢迎来到这篇关于使用 `ERNIEKit` 对 `ERNIE 4.5-0.3B` 模型进行预训练的教程！在当前的AI浪潮中，大语言模型 (Large Language Models, LLMs) 无疑是最耀眼的明星之一。百度文心（ERNIE）系列模型作为国内领先的大模型，以其卓越的性能和持续的技术创新而备受瞩目。

### 1.1 什么是大语言模型预训练 (PT)？

想象一下我们是如何学习语言的。在正式学习语法规则和词汇含义之前，我们会先大量接触和模仿周围人的说话方式。大语言模型的预训练 (Pre-training, PT) 与此类似。

简单来说，**预训练是指在大规模无标签文本数据上训练模型，让模型学习语言的内在规律、语法结构、语义信息以及世界知识的过程。** 这个阶段的模型并不针对某一个特定任务（如下游的问答、翻译），而是致力于构建一个通用的语言理解和生成能力的基础。这就像是为模型打下一个坚实的语言基础，使其后续能更好地适应各种具体的NLP任务。

预训练的核心任务通常是"预测下一个词" (Next Token Prediction)。给定一段文本，模型需要预测接下来最可能出现的词是什么。通过在海量文本上不断进行这样的预测练习，模型逐渐学会了词语之间的搭配关系、句法结构，甚至一些常识和逻辑。

### 1.2 为什么选择 ERNIE 4.5-0.3B 和 ERNIEKit？

在深入实践之前，让我们先花些时间，将 `ERNIE 4.5` 模型和 `ERNIEKit` 工具套件放置在当前主流的开源生态中进行一个横向对比。这能帮助我们更好地理解它们各自的特点和优势。

#### 1.2.1 ERNIE 4.5 模型家族概览

`ERNIE 4.5` 并不仅仅指一个模型，而是百度推出的一个覆盖从轻量级到千亿级参数的、包含多种规格的庞大模型家族。为了更好地理解我们即将使用的 `0.3B` 模型的定位，让我们先看一下 `ERNIE 4.5` 家族的全貌：

| 模型类别 | 模型名称 (部分) | 参数规模 (总参数/激活参数) | 主要特点 | 适用场景 |
| :--- | :--- | :--- | :--- | :--- |
| MoE 大语言模型 | `ERNIE-4.5-300B-A47B` | ~300B / 47B | 旗舰级纯语言模型，性能顶尖 | 要求极致性能的复杂任务 |
| MoE 大语言模型 | `ERNIE-4.5-21B-A3B` | ~21B / 3B | 高性能、高性价比的 MoE 模型 | 需要强大能力但资源受限的场景 |
| 多模态大模型 | `ERNIE-4.5-VL-424B-A47B`| ~424B / 47B | 支持文本、图像、视频输入 | 复杂的图文视频理解与生成 |
| **稠密语言模型** | **`ERNIE-4.5-0.3B`** | **3亿 (0.3B)** | **轻量级、资源友好、架构标准** | **学习、实验、快速验证、低资源部署** |

*注：MoE (Mixture-of-Experts) 是一种先进的模型架构，它拥有巨大的总参数量，但在处理每个输入时只激活一部分参数，从而实现高效推理。稠密模型 (Dense Model) 则会在每次计算时动用所有参数。*

#### 1.2.2 为什么以 ERNIE 4.5-0.3B 为例？

通过上表，我们可以清晰地看到 `ERNIE 4.5-0.3B` 在家族中的独特定位。在本教程中，我们选择它作为主角，主要基于以下几点考量：

1.  **为学习和实验而生**：与动辄需要大型集群才能训练的 MoE 巨型模型不同，`0.3B`（3亿）的参数量是专为开发者学习、研究和快速实验设计的。它的资源消耗“亲民”，让我们能在个人电脑或单台服务器上完成从预训练到微调的全过程。
2.  **极佳的资源友好性**：我们可以在单张消费级显卡（如 RTX 3090/4090, 24GB 显存）上运行本教程的完整流程。这极大地降低了学习大模型技术的硬件门槛。
3.  **标准架构的代表性**：作为一个标准的稠密（Dense）Transformer 模型，`ERNIE 4.5-0.3B` 的内部结构更易于理解，没有 MoE 等复杂组件。吃透了它的训练过程，就掌握了绝大多数标准大模型的核心原理，为将来理解更复杂的模型打下坚实基础。
4.  **官方工具链的完整支持**：`ERNIEKit` 为包括 `0.3B` 在内的全系列模型提供了一致的、工业级的工具链支持。用它来学习 `0.3B` 模型，就相当于在“专业级的驾校”里“开教练车”，能平滑地将所学技能迁移到未来操作更大、更复杂的模型上。

因此，`ERNIE 4.5-0.3B` 是我们进入文心大模型世界的理想“第一站”。

#### 1.2.3 主流开发工具套件对比

有了好的模型，还需要称手的工具。大模型的开发工具套件负责处理数据、驱动训练、执行推理等一系列复杂流程。

| 特性 | **ERNIEKit** | **Hugging Face (transformers+trl)** | **LLaMA-Factory** |
| :--- | :--- | :--- | :--- |
| **开发者/社区** | 百度 (Baidu) | Hugging Face Community | hiyouga (Community) |
| **核心框架** | PaddlePaddle | PyTorch, TensorFlow, JAX | PyTorch |
| **主要功能** | 全生命周期：预训练、SFT、DPO、LoRA、量化(QAT)、部署 | 核心组件库，支持几乎所有模型和训练方法，需编写较多代码 | 一站式微调框架，配置驱动，支持多种高效微调算法 |
| **易用性** | 工业级，命令行+配置文件，提供WebUI，学习曲线中等 | 非常灵活，但需要较强的编码能力，学习曲线较陡 | 对新手极其友好，提供WebUI，配置简单，学习曲线平缓 |
| **模型支持** | 专注于 ERNIE 系列模型，提供深度优化 | 支持最广泛的开源模型（Model Hub） | 支持主流的 LLaMA, Qwen, ChatGLM 等多种模型 |
| **适用场景** | ERNIE 模型深度开发、工业级应用、PaddlePaddle 技术栈 | 通用模型研究、算法实验、需要高度定制化的任务 | 快速上手、个人开发者、教学、高效微调实验 |

#### 1.2.4 为什么选择 ERNIEKit？

正如其名，`ERNIEKit` 是为 ERNIE 模型家族量身打造的官方工具套件。选择它，意味着：

*   **最佳适配与性能**：ERNIEKit 对 ERNIE 系列模型的支持是原生和最深入的。它提供了针对性的优化，能够最大化地发挥模型在飞桨（PaddlePaddle）框架上的训练和推理性能。
*   **工业级完整流程**：它不仅仅是一个微调工具，而是覆盖了从数据处理、预训练、监督微调（SFT）、偏好对齐（DPO）、参数高效调整（LoRA）到量化感知训练（QAT）和部署的全链路能力。学习 ERNIEKit 有助于理解大模型工业化落地的完整图景。
*   **官方维护与支持**：作为官方工具，其迭代更新会与 ERNIE 模型的最新进展保持同步，能获得最及时、最权威的技术支持。

通过本教程，我们将以 `ERNIE 4.5-0.3B` 为例，使用 `ERNIEKit` 中提供的底层预训练脚本，探索预训练的奥秘。这套组合拳（**官方模型 + 官方工具**）是深入学习百度文心大模型技术体系的最佳路径。即使我们使用的是一个相对较小的模型和数据集，其核心原理和流程对于理解更大模型的预训练也是相通的。

### 1.3 本教程的目标读者和学习成果

本教程主要面向：

*   对大语言模型预训练感兴趣的初学者。
*   希望了解如何在 ERNIEKit (PaddlePaddle) 框架下进行模型预训练的开发者。
*   有一定 Python 和深度学习基础，并希望动手实践 LLM 的学习者。

通过本教程，您将能够：

*   理解大语言模型预训练的基本概念和意义。
*   掌握使用 ERNIEKit 进行 ERNIE 4.5 模型预训练的完整流程。
*   了解预训练数据的准备方法。
*   学会如何配置 `yaml` 文件并启动预训练任务。
*   对预训练结果有一个初步的分析。
*   为后续进行模型微调 (Fine-tuning) 和特定任务应用打下基础。

让我们开始这段激动人心的学习之旅吧！

## 2. 环境准备

在开始我们的预训练之旅之前，我们需要确保拥有正确的开发环境。这主要包括安装核心的深度学习框架 PaddlePaddle 和 ERNIEKit 工具库。

### 2.1 安装 PaddlePaddle 和 ERNIEKit

ERNIEKit 对环境有一定要求，**强烈建议使用官方推荐的 Docker 镜像**，以避免环境不一致带来的问题。如果无法使用 Docker，请确保您的本地环境满足[官方文档的先决条件](https://github.com/PaddlePaddle/ERNIE/blob/develop/docs/erniekit.md#21-prerequisites) (如 CUDA >= 12.3, Python 3.10+ 等)。

**1. 克隆 ERNIE-develop 仓库**

ERNIEKit 的代码和所有脚本都包含在 `ERNIE-develop` 仓库中。首先，我们需要将其克隆到本地。

In [1]:
# git clone https://github.com/PaddlePaddle/ERNIE.git
%cd ERNIE-develop

/home/aistudio/ERNIE-develop


**2. 安装依赖**

ERNIEKit 所需的依赖项都列在 `requirements/gpu/requirements.txt` 文件中。

In [ ]:
!python -m pip install -r requirements/gpu/requirements.txt

**验证安装**：

安装完成后，您可以运行以下代码来验证 PaddlePaddle 是否成功安装并能正确识别您的 GPU。

In [5]:
import paddle

# 检查GPU是否可用
try:
    paddle.utils.run_check()
    print("PaddlePaddle GPU is available!")
except Exception as e:
    print(f"PaddlePaddle GPU check failed: {e}")
    print("If you intended to use GPU, please check your CUDA setup and PaddlePaddle installation.")

/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:715: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.
PaddlePaddle GPU is available!


I0717 10:30:30.392132   294 pir_interpreter.cc:1524] New Executor is Running ...
W0717 10:30:30.393514   294 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 12.8, Runtime API Version: 12.6
I0717 10:30:30.394114   294 pir_interpreter.cc:1547] pir interpreter is running by multi-thread mode ...


至此，我们的环境准备工作就完成了！接下来，我们将一起准备用于预训练的数据。

## 3. 数据准备

大语言模型之所以"大"，不仅仅在于其参数量，更在于其训练所依赖的海量数据。预训练阶段的数据是模型学习语言知识的源泉。

### 3.1 预训练数据格式 (.bin / .idx)

为了高效地加载和处理大规模文本数据，`ERNIEKit` 和 `PaddleNLP` 采用了标准的二进制 `mmap` (memory-mapped file) 格式。

*   **.bin 文件**：这是一个二进制文件，它存储了所有文本数据经过 Tokenizer（分词器）处理后转换得到的 Token ID 序列。简单来说，就是把所有的文字都变成了数字。
*   **.idx 文件**：这是一个索引文件，它记录了 `.bin` 文件中每条训练样本（通常是一篇文章或一个文档）的起始位置和长度。这样，在训练时可以快速定位到任意一条数据。

这种格式的数据组织方式有以下优点：

*   **高效读取**：可以直接在内存中映射文件内容，避免了大量的磁盘I/O操作。
*   **节省内存**：不需要一次性将所有数据都加载到内存中，可以按需读取。


### 3.2 获取预训练数据

为了方便用户快速开始，PaddleNLP 官方提供了一个处理好的 OpenWebTextCorpus 数据集子集，包含约10万篇文章。我们将使用这个数据集作为本教程的示例。

In [6]:
# 在 ERNIE-develop 目录下创建 data 文件夹
!mkdir data

In [8]:
# 下载数据文件
!wget https://bj.bcebos.com/paddlenlp/models/transformers/llama/data/llama_openwebtext_100k.bin -P ./data/
!wget https://bj.bcebos.com/paddlenlp/models/transformers/llama/data/llama_openwebtext_100k.idx -P ./data/

--2025-07-17 10:32:02--  https://bj.bcebos.com/paddlenlp/models/transformers/llama/data/llama_openwebtext_100k.bin
Resolving bj.bcebos.com (bj.bcebos.com)... 100.67.184.196, 100.64.80.160, 100.67.184.48
Connecting to bj.bcebos.com (bj.bcebos.com)|100.67.184.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 246736546 (235M) [application/octet-stream]
Saving to: './data/llama_openwebtext_100k.bin'

llama_openwebtext_1 100%[===================>] 235.31M   123MB/s    in 1.9s    

2025-07-17 10:32:04 (123 MB/s) - './data/llama_openwebtext_100k.bin' saved [246736546/246736546]

--2025-07-17 10:32:04--  https://bj.bcebos.com/paddlenlp/models/transformers/llama/data/llama_openwebtext_100k.idx
Resolving bj.bcebos.com (bj.bcebos.com)... 100.67.184.48, 100.64.80.160, 100.67.184.196
Connecting to bj.bcebos.com (bj.bcebos.com)|100.67.184.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2000042 (1.9M) [application/octet-stream]
Saving to: './da

下载完成后，您的 `./data/` 目录下应该会有两个文件：
*   `llama_openwebtext_100k.bin`
*   `llama_openwebtext_100k.idx`

**重要提示：** 这些数据文件虽然最初是为 LLaMA 模型准备的，但其 `mmap` 格式是通用的，同样适用于 ERNIE 等模型的预训练。在实际训练时，我们会使用 ERNIE 4.5 的 Tokenizer 来处理这些文本数据转换成的 token ID。

### 3.3 (可选) 使用自定义数据

如果您想使用自己的文本数据，需要将其转换为 `.bin` 和 `.idx` 格式。`PaddleNLP` 仓库（注意不是 ERNIE-develop 仓库）中提供了数据预处理脚本 `llm/tools/preprocess_data.py`。

大致流程如下：
1.  **收集** 您的原始文本数据（如 `.txt` 或 `.jsonl` 文件）。
2.  **运行脚本**：使用 `preprocess_data.py` 脚本，指定模型对应的 `tokenizer_name_or_path`（例如，应使用 `PaddlePaddle/ERNIE-4.5-0.3B-Paddle` 的 tokenizer），将您的文本文件转换为 `.bin` 和 `.idx` 文件。
    ```bash
    # 示例命令 (需在 PaddleNLP 仓库下运行，并安装其依赖)
    # python llm/tools/preprocess_data.py \
    #     --model_name_or_path PaddlePaddle/ERNIE-4.5-0.3B-Paddle \
    #     --input_path /path/to/your/text_files.list \
    #     --output_prefix /path/to/your/output/my_data
    ```
详细用法请参考 PaddleNLP 仓库中的相关文档。对于本教程，我们直接使用已下载好的数据。

现在数据已经准备好了，下一步我们将简要介绍一下大模型预训练的基本原理。


## 4. 模型预训练原理简介

在我们动手运行代码之前，花一些时间了解预训练背后的基本原理是非常有益的。


### 4.1 核心思想：Next Token Prediction (下一个词元预测)

大语言模型预训练最核心、最经典的任务就是 **Next Token Prediction**，也叫做 Causal Language Modeling (CLM，因果语言建模)。

**简单来说，就是给定一段文本序列，模型的目标是预测这个序列中的下一个词元 (Token) 是什么。**

**举个例子：**

假设我们有这样一句话："今天天气真不错，我们一起去公园"

1.  当模型看到：`["今天"]` -> 它需要预测：`"天气"`
2.  当模型看到：`["今天", "天气"]` -> 它需要预测：`"真"`
3.  当模型看到：`["今天", "天气", "真"]` -> 它需要预测：`"不错"`

模型会接触海量的文本数据，在这些数据上不断地进行这种"猜词游戏"。通过比较自己的预测和真实文本中的下一个词，模型会计算出一个"错误程度"（即损失 Loss），然后根据这个错误来调整内部的参数（权重），力求下次猜得更准。

这个过程虽然简单，但当数据量足够大、模型参数足够多时，模型为了降低预测错误，就必须学会：

*   **词汇知识**：哪些词经常一起出现。
*   **语法结构**：主谓宾、时态等。
*   **语义连贯性**：上下文应该如何衔接。
*   **常识知识**：例如，"天空是蓝色的"。


### 4.2 模型架构：Transformer

目前几乎所有主流的大语言模型，包括 ERNIE 4.5，都基于 **Transformer** 架构。对于 Next Token Prediction 任务，通常使用的是 Transformer 的 **Decoder-Only** (仅解码器) 架构。

其核心组件包括：
1.  **Embedding Layer (嵌入层)** ：将输入的 Token ID 转换为能捕捉语义的向量。
2.  **Positional Encoding (位置编码)** ：为模型注入 Token 在序列中的位置信息。
3.  **Multi-Head Self-Attention (多头自注意力机制)** ：Transformer 的核心，允许模型在处理一个 Token 时，动态地关注序列中所有其他 Token 的信息。
4.  **Feed-Forward Network (前馈神经网络)** ：进行进一步的非线性变换，增强模型的表达能力。
5.  **Layer Normalization 和 Residual Connections** ：稳定训练过程，使训练更深的网络成为可能。

通过堆叠多个这样的 Transformer Block (例如 ERNIE 4.5-0.3B 有32层)，模型就能够学习到非常复杂的语言模式。

### 4.3 损失函数：Cross-Entropy (交叉熵)

在模型预测出下一个词元的概率分布后，我们使用**交叉熵损失 (Cross-Entropy Loss)** 来衡量预测的好坏。它会比较**模型预测的概率分布**和**真实的下一个词元**之间的"差异"。训练的目标就是通过不断调整模型的参数，使得在整个训练数据集上的总交叉熵损失最小化。

了解了这些基本原理后，我们就可以更有信心地开始实际操作了！


## 5. 开始预训练

理论知识武装完毕，数据也准备就绪，现在是时候动手实践，启动我们的第一个 ERNIE 4.5 预训练任务了！

**重要前提：**
1.  请确保您已按照 **"2. 环境准备"** 部分的说明，安装了所有依赖。
2.  请确保您已按照 **"3. 数据准备"** 部分的说明，下载了数据文件并放置在 `./data/` 目录下。
3.  **工作目录**：后续的所有命令，都假设您当前的终端工作目录是 `ERNIE-develop`。


### 5.1 预训练脚本和配置文件

我们知道 `ERNIEKit` 中执行预训练的核心脚本是 `./examples/pre-training/ernie/pretrain`。我们需要通过一个 `yaml` 配置文件来告诉这个脚本使用哪个模型、用什么数据、以及如何进行训练。

原始仓库中的 `pretrain_96_gpus.yaml` 是为千亿级 MoE 模型在大型集群上设计的，无法直接使用。我们需要创建一个简化版的配置文件，专门用于在单张 GPU 上训练 `ERNIE 4.5-0.3B` 模型。

**创建配置文件 `pretrain_ernie_0.3b_demo.yaml`**

请在 `ERNIE-develop` 目录下创建一个新的 `yaml` 文件，命名为 `pretrain_ernie_0.3b_demo.yaml`，并将以下内容复制进去：

```yaml
# 适用于 ERNIE 4.5-0.3B 单卡预训练的示例配置文件

# ---------------------------model args-------------------------------------------------#
model_args:
    model_name_or_path: "../../../data/models/30654/ERNIE-4.5-0.3B-Base-Paddle" # 指定模型名称，ERNIEKit会尝试从AI Studio或HuggingFace下载
    tokenizer_name: "../../../data/models/30654/ERNIE-4.5-0.3B-Base-Paddle" # 指定分词器
    output_dir: ./output_pretrain/ # 训练过程中模型权重、日志等的输出目录
    max_seq_length: 1024 # 模型处理的最大序列长度，可根据显存调整

# ---------------------------trainer args-------------------------------------------------#
trainer_args:
    input_dir: "1.0 ../../data/llama_openwebtext_100k 1.0 ../../data/llama_openwebtext_100k" # 训练和验证数据集的权重及路径
    split: "998,1,1" # 训练、验证、测试集划分比例

    do_train: True
    dataloader_num_workers: 0 # Windows下建议设为0，Linux可适当调大
    disable_tqdm: True
    logging_steps: 10 # 每隔10步打印一次日志
    eval_steps: 100 # 每隔100步进行一次评估
    save_steps: 200 # 每隔200步保存一次模型权重 (checkpoint)
    max_steps: 400 # 最大训练步数，用于快速演示
    
    # --- 学习率与优化器 ---
    adam_beta1: 0.9
    adam_beta2: 0.95
    adam_epsilon: 1e-8
    learning_rate: 1e-4
    min_lr: 1e-5
    lr_scheduler: "wsd_cosine" # 使用带预热的余弦退火学习率策略
    max_grad_norm: 1.0
    weight_decay: 0.1
    warmup_steps: 50 # 学习率预热步数
    
    # --- 批大小与显存 ---
    gradient_accumulation_steps: 8 # 梯度累积步数
    per_device_train_batch_size: 1 # 每张卡的批大小，实际 batch_size = per_device_train_batch_size * gradient_accumulation_steps
    per_device_eval_batch_size: 2
    head_dim: 128 # 注意力头的维度大小，需要与预训练模型保持一致
    
    # --- 性能与精度 ---
    bf16: False # 是否启用 BF16，如果GPU不支持（如30/40系），请设为False
    fp16: True  # 是否启用 FP16 (混合精度) 训练，对30/40系GPU友好
    fp16_opt_level: "O1" # FP16 优化级别，O1 较稳定
    
    # --- 分布式参数 (单卡训练设为1或默认) ---
    moe_group: "dummy" # 对于非MoE模型，设置为dummy以禁用MoE相关逻辑
    pipeline_parallel_degree: 1
    tensor_parallel_degree: 1
    sharding: "" # 单卡训练不使用sharding
    
    # --- 其他 ---
    seed: 42
    save_total_limit: 2 # 最多保存多少个checkpoint
    overwrite_output_dir: true # 覆盖输出目录
```

**关键参数解释**：
*   `model_name_or_path`: 指定了我们要训练的模型。ERNIEKit 会自动处理下载。
*   `output_dir`: 所有训练产物（模型权重、日志）的保存位置。
*   `input_dir`: 指向我们下载的 `.bin` 和 `.idx` 数据文件。**注意**：这里不需要文件后缀，只需要路径。
*   `max_seq_length`: 模型能处理的文本最大长度。此值越大，显存占用越高。1024 对大部分24GB显存的显卡来说是安全的。
*   `max_steps`, `save_steps`, `logging_steps`: 控制训练的总步数、保存频率和日志打印频率。我们设置为较小的值以便快速看到结果。
*   `per_device_train_batch_size` & `gradient_accumulation_steps`: 这是控制**有效批大小 (Effective Batch Size)** 和**显存占用**的关键。`per_device_train_batch_size` 是单次前向传播的样本数，直接影响显存。通过累积 `gradient_accumulation_steps` 次梯度再更新一次模型，我们可以在不增加显存的情况下，达到 `1 * 8 = 8` 的有效批大小。
*   `fp16: True`: 启用混合精度训练，可以显著减少显存占用并加速训练，对于消费级显卡至关重要。

### 5.2 下载模型

在开始训练之前，我们需要先将 `ERNIE-4.5-0.3B-Paddle` 模型文件下载到本地。您可以使用 `aistudio-sdk` (推荐) 或 `huggingface-cli`。

**首先安装 aistudio-sdk:**

In [ ]:
!pip install --upgrade aistudio-sdk

**然后下载模型:**

In [ ]:
!aistudio download --model PaddlePaddle/ERNIE-4.5-0.3B-Paddle --local_dir baidu/ERNIE-4.5-0.3B-Paddle

下载完成后，我们的配置文件中指定的 `"baidu/ERNIE-4.5-0.3B-Paddle"` 路径就能被正确找到了。如果下载出现`time out`的同学，可以尝试重新下载，或者使用[本项目](https://aistudio.baidu.com/project/edit/9382861)挂载的模型：/home/aistudio/data/models/30654/ERNIE-4.5-0.3B-Base-Paddle。


本项目将采用挂在的模型进行演示！

### 5.3 启动单卡预训练

一切准备就绪后，我们可以在 `ERNIE-develop` 目录下执行以下命令来启动单卡预训练。

- /home/aistudio/ERNIE-develop/examples/pre-training/models/moe/token_dispatcher/fp8_utils.py，注释Fp8的导入
- /home/aistudio/ERNIE-develop/examples/pre-training/models/fp8_linear.py ， 注释Fp8的导入
- /home/aistudio/ERNIE-develop/examples/pre-training/models/ernie/modeling.py ,注释Fp8的导入


修复 MoE 参数错误 (如果遇到)

如果您在运行脚本时遇到 `KeyError: 'moe_group'` 等其他错误，这是因为脚本假设配置文件中一定有 `moe_group` 参数（这是为 MoE 模型设计的），但我们的简化配置中没有它（因为 `ERNIE-4.5-0.3B` 是稠密模型）。

**快速修复步骤**：
1. 打开文件 `ERNIE-develop/examples/pre-training/ernie/pretrain.py` (使用文本编辑器)。
2. 找到这一行 (大约第 394 行)：`if trainer_args["moe_group"].lower() in {"mp", "tp", "model", "dummy"}:`
3. 将其修改为：`moe_group = trainer_args.get("moe_group", None)  # 添加检查，避免 KeyError\nif moe_group and moe_group.lower() in {"mp", "tp", "model", "dummy"}:`
4. 在`/home/aistudio/ERNIE-develop/examples/pre-training/models/ernie/configuration.py`的ErnieMoEConfig类中（大约247行）加入`self.head_dim = None`
5. 在`/home/aistudio/ERNIE-develop/examples/pre-training/models/ernie/modeling_moe.py`的大约1502行加入（原因：官方代码在计算 RoPE 的 sin 和 cos 缓存时，使用了 float64 类型，而示例模型期望的是 float32）：
```
            cos_cached = np.cos(emb)[:, :].astype("float32")
            sin_cached = np.sin(emb)[:, :].astype("float32")
```
6. 在`/home/aistudio/ERNIE-develop/examples/pre-training/ernie/src/trainers/pretraining_trainer.py`大约1096行，删除：  (添加一个分布式环境的判断)
```python
# dist.all_reduce(tr_loss, dist.ReduceOp.SUM)
# tr_loss_scalar = tr_loss.item() / dist.get_world_size()

```
加入：
```python
            if self.args.world_size > 1:
                dist.all_reduce(tr_loss, dist.ReduceOp.SUM)
                tr_loss_scalar = tr_loss.item() / dist.get_world_size()
            else:
                tr_loss_scalar = tr_loss_single_dp_scalar

```

删除（大约1150行）
```python
dist.all_reduce(numel_tensor)
```
加入：

```python
                if self.args.world_size > 1:
                    dist.all_reduce(numel_tensor)
```

删除（大约1212行）：
```python

paddle.distributed.barrier()
```

加入:
```python
if self.args.world_size > 1:
                paddle.distributed.barrier()
```
6. 保存文件并重新运行启动命令。

这个修改是安全的，不会影响我们的训练，因为我们不使用 MoE 功能。

In [2]:
# 1. 首先，切换工作目录到 pre-training 文件夹
%cd ./examples/pre-training/

# 2. 然后，从此目录启动训练脚本
#    注意：配置文件的路径因为工作目录变化，需要使用 ../../ 来返回到项目根目录
!python -u ernie/pretrain.py --config ../../pretrain_ernie_0.3b_demo.yaml

/home/aistudio/ERNIE-develop/examples/pre-training
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:715: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
/home/aistudio/external-libraries/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
[WARNING] 2025-07-17 18:20:33,983 [ modeling.py:   75]:    Use flash attention in scaled-dot-product. Attention mask is deprecated
W0717 18:20:34.138628  2289 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability:

### 5.4 观察训练过程

当训练开始后，您会在终端看到类似下面的日志信息（具体数值可能不同）：

**关键日志解读：**
*   `global_step`: 当前的总训练步数。
*   `loss`: 当前这一步计算出的损失值。**这是非常重要的指标！** 我们期望看到 `loss` 随着训练的进行而逐渐下降。
*   `learning_rate`: 当前的学习率。您可以看到它在 `warmup_steps` 内逐渐增加。
*   `speed`: 训练速度，表示每秒处理多少步。
*   `vram`: 训练过程中占用的显存（MB）。

**预期现象：**
*   **Loss 下降**：这是最重要的，表明模型正在从数据中学习。
*   **显存占用**：对于 `ERNIE-4.5-0.3B` 模型，在 上述配置下，显存占用预计在 10GB 左右范围。如果出现 `CUDA out of memory` 错误，说明显存不足，需要调整配置（见后续"进阶与常见问题"部分）。

## 6. 结果分析与解读

当训练达到您在配置文件中设定的 `max_steps` 后，预训练过程就告一段落了。所有的训练输出，都会保存在配置文件 `output_dir` 指定的目录下，即 `./output_pretrain/`。

训练完成后，您可以查看这个目录的内容：

In [3]:
ls -lh ./output_pretrain/

total 1.6G
-rw-r--r-- 1 aistudio aistudio  23K Jul 17 18:24 added_tokens.json
-rw-r--r-- 1 aistudio aistudio  179 Jul 17 18:24 all_results.json
drwxr-xr-x 1 aistudio aistudio 4.0K Jul 17 17:25 checkpoint-200/
drwxr-xr-x 2 aistudio aistudio 4.0K Jul 17 18:24 checkpoint-400/
-rw-r--r-- 1 aistudio aistudio 4.1K Jul 17 18:24 config.json
-rw-r--r-- 1 aistudio aistudio  167 Jul 17 18:24 generation_config.json
drwxr-xr-x 1 aistudio aistudio 4.0K Jul 17 15:47 log/
-rw-r--r-- 1 aistudio aistudio 1.6G Jul 17 18:24 model_state.pdparams
drwxr-xr-x 1 aistudio aistudio 4.0K Jul 17 18:20 runs/
-rw-r--r-- 1 aistudio aistudio  16K Jul 17 18:24 special_tokens_map.json
-rw-r--r-- 1 aistudio aistudio 9.8K Jul 17 18:24 static_name_to_dyg_name.json
-rw-r--r-- 1 aistudio aistudio 1.6M Jul 17 18:24 tokenizer.model
-rw-r--r-- 1 aistudio aistudio 132K Jul 17 18:24 tokenizer_config.json
-rw-r--r-- 1 aistudio aistudio  179 Jul 17 18:24 train_results.json
-rw-r--r-- 1 aistudio aistudio  322 Jul 17 18:24 trainer_st

**关键文件和目录解释：**
*   **根目录文件**：保存的是训练完成时（达到 `max_steps`）的最终模型和配置。
*   **`checkpoint-<step_number>/`**: 这些是根据 `save_steps` 参数定期保存的检查点目录。每个检查点都包含当时的完整模型状态，可以用于从该点恢复训练，或者加载该阶段的模型进行评估。
*   **`visualdl/`**: VisualDL 的日志文件，可以使用 `visualdl --logdir ./output_pretrain/visualdl` 来启动一个Web服务，可视化地查看 loss 曲线等指标。

**初步评估预训练效果：**
对于我们这个简短的教程，主要关注 **`loss` 是否有明显的下降趋势**。例如，从初始的 9-10 下降到 6-7 的区间，就可以认为基本的预训练流程是成功的，模型学到了一些语言的统计规律。要获得一个真正强大的模型，需要在更大规模的数据集上进行更长时间的训练。

## 7. 进阶与常见问题

### 7.1 显存不足 (CUDA out of memory)

这是最常见的问题。解决方案：
1.  **增大梯度累积步数**：在配置文件中进一步增大 `gradient_accumulation_steps` (例如 16, 32)。这是最有效且推荐的方法。
2.  **减小批大小**：将 `per_device_train_batch_size` 设为更小的值（但通常已经是1了）。
3.  **减小序列长度**：减小 `max_seq_length` (例如 512)，但这会影响模型学习长距离依赖的能力。
4.  **使用更激进的优化**：`ERNIEKit` 支持梯度重计算（Recompute）等更高级的显存优化技术，可以在配置文件中开启 `recompute: True`（可能在 `model_args`下），但这会增加计算时间。

## 8. 总结与展望

恭喜您！通过本教程，您已经：

*   了解了使用 `ERNIEKit` 对 `ERNIE 4.5` 模型进行预训练的完整流程。
*   掌握了如何准备数据、配置 `yaml` 文件以及启动和监控训练。
*   对预训练的核心概念、关键参数和结果分析有了深入的理解。

本次教程的核心是，我们通过分析 `ERNIEKit` 的底层脚本，成功地将一个为大型集群设计的复杂预训练流程，简化为了一个可以在单张消费级显卡上运行的、适合学习和实验的迷你流程。

**展望未来：**
预训练得到的模型是一个通用的"语言底座"。接下来，您可以探索：

*   **更大规模的预训练**：在更多的数据上，训练更长的时间，以获得性能更强的基础模型。
*   **模型微调 (Fine-tuning)** ：这通常是预训练之后的下一步。使用有监督的数据（如问答对）对预训练好的模型进行微调（SFT），可以使其更好地遵循指令，完成特定任务。`ERNIEKit` 提供了强大的 `erniekit train` 命令来支持 SFT 和 DPO 等微调方法。

希望本教程能为您打开探索文心（ERNIE）大模型世界的大门。感谢您的阅读！

# 问题反馈/与我联系： Wechat：G_Fuji